In [1]:
# from retriever import Retriever
import torch 
from evaluate import load
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from ast import literal_eval
from statistics import mean, median, quantiles
import pickle
from utils import SEP_TOKEN, init_tokenizer, init_prompt_model, load_model
from prompt_str import SYSTEM_PROMPT, USER_PROMPT, CONTEXT_PROMPT, SUFFIX_PROMPT, PREFIX_PROMPT
from gnd_dataset import GNDDataset
from gnd_graph import GNDGraph
from data_collator import DataCollator, GraphDataCollator
from retriever import Retriever
from transformers import  pipeline, AutoModel
import yaml
from tqdm import tqdm
import torch_geometric as pyg
from datasets import Dataset
import networkx as nx
import numpy as np


/home/katja/projects/xmlc-knowledge/ki-env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = "cuda:1" if torch.cuda.is_available() else "cpu"

In [3]:
data_path = "datasets/no_context/"

In [4]:
gnd = pickle.load(open("data/gnd.pickle", "rb"))
index = pickle.load(open("data/label_index.pkl", "rb"))
mapping = pickle.load(open("data/label_mapping.pkl", "rb"))

config = yaml.safe_load(open("dev_cfg/cfg.yml", "r"))

In [5]:
gnd = GNDGraph(gnd)

In [6]:
ds = GNDDataset(
    data_dir=data_path,
    gnd_graph=gnd, 
    load_from_disk=True,
    config=config,
)

In [7]:
data_dict = {
    "label": [],
    "neighbors": [],
}
for node in gnd.nodes(data=True):
    idn, data = node
    name = gnd.pref_label_name(idn)
    neighbors = [gnd.pref_label_name(node_idn) for node_idn in gnd.neighbors(idn)]
    if neighbors:
        data_dict["label"].append(name)
        data_dict["neighbors"].append(neighbors)

In [8]:
ds = Dataset.from_dict(data_dict)

In [9]:
tokenizer = init_tokenizer(config["model_name"])

In [29]:
ds_train = ds.train_test_split(test_size=0.05, seed=42, shuffle=True)

collator = GraphDataCollator(
    tokenizer=tokenizer,
    config=config,
    device=device,
)
dl = DataLoader(
    ds,
    batch_size=256,
    collate_fn=collator,
    shuffle=False,
)

In [31]:
ds_train

DatasetDict({
    train: Dataset({
        features: ['label', 'neighbors'],
        num_rows: 247563
    })
    test: Dataset({
        features: ['label', 'neighbors'],
        num_rows: 13030
    })
})

In [22]:
trunc = 0
for i in tqdm(dl):
    longer = (i["seq_lengths"] >= 100).sum().item()
    trunc += longer

100%|██████████| 1018/1018 [00:29<00:00, 33.94it/s]


In [24]:
trunc/len(dl.dataset)

0.002866539009106154

In [ ]:
index_path = config["context"].get("index_path")
mapping_path = config["context"].get("mapping_path")
ret = Retriever(
    retriever_model=config["sentence_transformer_model"],	
    graph=gnd,
    device=device,
)
ret.load_search_index(
    index_path=index_path,
    mapping_path=mapping_path,
)

<faiss.swigfaiss_avx512.IndexHNSWFlat; proxy of <Swig Object of type 'faiss::IndexHNSWFlat *' at 0x7f8011b965e0> >

In [8]:
tokenizer = init_tokenizer(config["model_name"])

In [10]:
dc = DataCollator(
    tokenizer=tokenizer,
    graph=gnd,
    config=config,
    device=device,
)

In [11]:
dl = DataLoader(
    ds["validate"],
    batch_size=1,
    shuffle=False,
    collate_fn=lambda x: dc(x, inference=True),
)

In [7]:
val_ds = ds["validate"]

In [9]:
# nx.draw(G, with_labels=True, node_size=500, font_size=10)

In [14]:
model = AutoModel.from_pretrained(config["model_name"]).to(device)

Loading checkpoint shards: 100%|██████████| 2/2 [00:08<00:00,  4.05s/it]


In [15]:
data = pyg.utils.from_networkx(G)
pref = [p[0] for p in data.pref]
tok_data = tokenizer(pref, padding="max_length", truncation=True, max_length=8, return_tensors="pt", add_special_tokens=False)["input_ids"]
data.x = model.embed_tokens(tok_data.to(device)).mean(dim=1).detach().cpu() 

In [16]:
data.x.shape

torch.Size([10, 3072])

In [17]:
max_nodes = 0
avg_n_nodes = []
data_g = []
for instance in tqdm(ds["validate"]):
    i_graph = instance["context_graph"][0]
    G = nx.node_link_graph(i_graph, edges="edges") 
    G.add_node("title", pref=example, alt=None, hidden=None, type=None, gnd_id=None)
    for node in G.nodes:
        if node != "title":
            G.add_edge(node, "title", relation="context", weight=1.0)
    pg_data = pyg.utils.from_networkx(G)
    data_g.append(pg_data)
    avg_n_nodes.append(pg_data.num_nodes)
    if pg_data.num_nodes > max_nodes:
        max_nodes = pg_data.num_nodes

100%|██████████| 2200/2200 [00:03<00:00, 596.10it/s]


In [19]:
l = pyg.loader.DataLoader(
    data_g, 
    batch_size=2, 
    shuffle=False, 
    follow_batch=["pref"],
    num_workers=0,
)

In [20]:
for data in l:
    data = data.to(device)
    print(data)
    break

DataBatch(edge_index=[2, 23], alt=[2], gnd_id=[2], hidden=[2], pref=[2], pref_batch=[2], pref_ptr=[3], type=[2], relation=[2], weight=[23], num_nodes=15, batch=[15], ptr=[3])


In [56]:
ds["validate"].add_column("pyg", data_g)

ArrowInvalid: Could not convert Data(edge_index=[2, 10], alt=[7], gnd_id=[7], hidden=[7], pref=[7], type=[7], relation=[10], weight=[10], num_nodes=7) with type Data: did not recognize Python value type when inferring an Arrow data type

In [21]:
max_nodes, mean(avg_n_nodes)

(92, 8.198636363636364)

In [22]:
gat = pyg.nn.GAT(
    in_channels=3072,
    out_channels=3072,
    hidden_channels=1024,
    num_layers=2,
)
gat = gat.to(device)

In [22]:
data.edge_index

tensor([[ 0,  0,  0,  1,  2,  2,  3,  3,  4,  4,  5,  5,  6,  6,  7,  8,  9,  9,
          9, 10, 10, 11, 11, 11, 11, 12, 12, 12, 13, 13, 13, 13, 13],
        [10,  8, 14, 14,  4, 14, 13, 14, 13, 14, 13, 14, 11, 14, 14, 14, 13,  7,
         14,  0, 14, 13,  1,  6, 14,  0,  2, 14,  1,  9, 11,  5, 14]])

In [19]:
gat(data.x.to(device), data.edge_index.to(device)).shape

torch.Size([15, 3072])

In [7]:
model_name = "meta-llama/Llama-3.2-3B-Instruct"

In [11]:
pipe = pipeline(
    "text-generation",
    model=model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [20]:
raw_predictions = []
count = 0
for row in ds["validate"]:
    title = row["title"]
    context = ", ".join(row["context_str"])
    messages = [
    {"role": "system", "content": CONTEXT_PROMPT.format(context)},
    {"role": "user", "content": USER_PROMPT.format(title=title)},
    ]
    outputs = pipe(messages, num_return_sequences=1, do_sample=True, temperature=0.7)
    new_tokens = outputs[0]["generated_text"][-1]["content"]
    raw_predictions.append(new_tokens.strip())
    print(title)
    print(row["label-names"])
    print(new_tokens.strip())
    count += 1
    if count >= 1000:
        break
    print("-"* 50)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Synökologische Studien zum simultanen Befall von Winterweizen (Triticum aestivum L.) mit Aphiden und getreidepathogenen Pilzen
['Ernteertrag', 'Phytopathogene Pilze', 'Winterweizen', 'Synökologie', 'Getreideblattläuse']
Synökologische Studien, Winterweizen, Aphiden, Pilze, Get
--------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Regeln für die Ansetzung von Körperschaftsnamen RAK-Körperschaften ; Bestimmungen d. RAK-WB, Grundbestimmungen d. RAK-ÖB, GKD-Erl. u. -Anh.
['Körperschaft', 'Formalerschließung']
RAK-Körperschaften, Körperschaftsrecht, Deutschland,
--------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Die Mikrokontroller 8051, 8052 und 80C517
['Mikrocontroller 8051', 'Mikrocontroller 80C517', 'Mikrocontroller 8052']
Mikrocontroller, 8051, 8052, 80C517, Mikro
--------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Wie man durch Manipulation und Verdrehung der Tatsachen Legenden erzeugt e. Auseinandersetzung mit Karl Marx u. seinem Hauptwerk "Das Kapital"
['Rezeption', 'Marx, Karl : Das Kapital', 'Marx, Karl (1818-1883)']
- Legendenbildung
- Manipulation
- Propaganda
- Falschdarstellung
--------------------------------------------------


KeyboardInterrupt: 

In [4]:
retriever = Retriever(retriever_model="BAAI/bge-m3", device=device)

In [6]:
pred_df = pd.read_csv("results/prompt-tuning-text-context-3-label/predictions-test-checkpoint-28000.csv")
# "results/Llama-3.2-3B-Instruct-hard-prompt/predictions.csv"
# "results/prompt-tuning-baseline/predictions-test-checkpoint-28000.csv"
pred_df["predictions"] = pred_df["predictions"].apply(literal_eval)
pred_df["label-ids"] = pred_df["label-ids"].apply(literal_eval)
pred_df["label-names"] = pred_df["label-names"].apply(literal_eval)

In [16]:
pred_df.sample(5)

,predictions,raw_predictions,label-ids,label-names,title,reranked-predictions
4576,"[041358899, 040013073, 041969103]",Alltag; Neue Medien; Selbstbestimmung,"[040718085, 04120719X, 041275764, 041761928, 0...","[Neue Medien, Medienkonsum, Medienkompetenz, F...",Die digitale Verführung Selbstbestimmt leben ...,"['041358899', '041969103', '040013073']"
5431,"[040410056, 04049716X, 958816999, 971787190, 0...",Rezeption; Griechisch; Mythologie; Odyssea; Po...,"[040418626, 040541460, 041298918, 042896347, 9...","[Neuplatonismus, Allegorese, Seele, Porphyrius...","Die Seele im Kosmos Porphyrios, Über die Nymp...","['040410056', '958816999', '971787190', '04113..."
8248,"[041229266, 04126892X, 040118827, 040097366]",Deutschland; Chancengleichheit; Bildungsforsch...,"[040058719, 040059626, 041212029, 041293169, 0...","[Berufsausbildung, Übergang (Sozialwissenschaf...","Übergänge nach der Schule als ""zweite Chance...","['041229266', '040097366', '04126892X', '04011..."
7655,[040487172],Computerarchitektur,[040487172],[Computerarchitektur],Rechnerarchitektur Einführung in den Aufbau m...,['040487172']
3585,"[040595552, 040118827]",Deutschland; Testament,"[040118827, 040178897, 040595552]","[Deutschland, Testament, Formvorschrift]",Neue ordentliche Testamentsformen,"['040595552', '040118827']"


In [48]:
import networkx as nx

def inverse_distance_weight(graph, gold_node, predicted_node):
    """
    Calculate graph distance weight between two nodes in a graph.
    The weight is inversely proportional to the shortest path distance between the nodes.

    Args:
        graph (networkx.Graph): The graph containing the nodes.
        gold_node (str): The node representing the gold standard.
        predicted_node (str): The node representing the predicted label.
    
    Returns:
        float: The weight based on the distance between the nodes.
    """
    weight = 0.0
    # Compute shortest path distance
    if gold_node == predicted_node:
        distance = 0  # Perfect match
    else:
        try:
            if gold_node not in graph or predicted_node not in graph:
                return 0.0
            distance = nx.shortest_path_length(graph, source=gold_node, target=predicted_node)
        except nx.NetworkXNoPath:
            distance = float('inf')  # No path exists

        # Weight inversely proportional to distance
    weight = 1 / (1 + distance) if distance != float('inf') else 0

    # Normalize by the number of predictions
    return weight

In [24]:
p = "040118894"
g = "040118827"

In [28]:
samples = pred_df.sample(10)

In [49]:
weight_prec = []
prec = []
for i, s in pred_df.iterrows():
    preds = s["predictions"]
    gold = s["label-ids"]
    weights = []
    correct = set(preds).intersection(set(gold))
    prec.append(len(correct) / len(preds) if preds else 0)
    for p in preds:
        max_weight = - float('inf')
        for g in gold:
            weight = graph_distance_weight(gnd, g, p)
            if weight > max_weight:
                max_weight = weight
        weights.append(max_weight)
    weighted_prec = sum(weights) / len(weights) if weights else 0
    weight_prec.append(weighted_prec)

In [50]:
mean(prec), mean(weight_prec)

(0.47784842722446713, 0.5259823097067122)